<a href="https://colab.research.google.com/github/teploe-odealko/meteoinfo_parsing/blob/main/news_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сводка word

In [1]:
!pip install python-docx
!pip install html2text

     |████████████████████████████████| 5.6 MB 6.4 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=cfb689c31c5bd85ca81fe2608af8a8b1324ee0aef88243346a88be69b70a0a55
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [5]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

Selecting previously unselected package chromium-codecs-ffmpeg-extra.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../chromium-codecs-ffmpeg-extra_91.0.4472.101-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-codecs-ffmpeg-extra (91.0.4472.101-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-browser.
Preparing to unpack .../chromium-browser_91.0.4472.101-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-browser (91.0.4472.101-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-browser-l10n.
Preparing to unpack .../chromium-browser-l10n_91.0.4472.101-0ubuntu0.18.04.1_all.deb ...
Unpacking chromium-browser-l10n (91.0.4472.101-0ubuntu0.18.04.1) ...
Selecting previously unselected package chromium-chromedriver.
Preparing to unpack .../chromium-chromedriver_91.0.4472.101-0ubuntu0.18.04.1_amd64.deb ...
Unpacking chromium-chromedriver (91.0.4472.101-0ubuntu0.18.04.1) ...
Setting up chromium-codecs-ffmp

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [7]:
import html2text

In [20]:
RBC_URL = 'https://www.rbc.ru/story/5e2881539a794724ab627caa'
wd.get(RBC_URL)
news_links = wd.find_elements_by_xpath("//a[@class='item__link']")
headers = wd.find_elements_by_xpath("//span[@class='item__title rm-cm-item-text']")

for i, header in enumerate(headers):
    try:
        if ' о пандемии на' in header.text.split('.')[1]:
            break
    except IndexError:
        continue

news_links[i].click()
elem = wd.find_element_by_xpath("//*")
source_code = elem.get_attribute("outerHTML")

h = html2text.HTML2Text()
h.ignore_links = True
source_text = h.handle(source_code)
rbc_news = [s[4:] for s in source_text.split('\n') if s.startswith('  * ')]

In [9]:
STOPCORONA_URL = 'https://стопкоронавирус.рф/news/'
wd.get(STOPCORONA_URL)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='cv-news-page__news-list-item']//h2")
stopcorona_headers = [header.text for header in stopcorona_headers_elements]


In [10]:
STOPCORONA_URL_BUISNESS = 'https://стопкоронавирус.рф/news/?tags=1'
wd.get(STOPCORONA_URL_BUISNESS)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='cv-news-page__news-list-item']//h2")
stopcorona_headers_buisness = [header.text for header in stopcorona_headers_elements]


In [11]:
YANDEX_URL = 'https://yandex.ru/news/rubric/koronavirus'
wd.get(YANDEX_URL)

stopcorona_headers_elements = wd.find_elements_by_xpath("//div[@class='mg-card__annotation']")
yandex_annotation = [header.text for header in stopcorona_headers_elements if header.text != '']


In [12]:
def paragraph_news(doc):
    doc.add_paragraph('Справочно')
    p = doc.add_paragraph()
    runner = p.add_run('РБК')
    runner.bold = True
    for news in rbc_news:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Стопкоронавирус')
    runner.bold = True
    for news in stopcorona_headers[:10]:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Стопкоронавирус по тегу "бизнесу"')
    runner.bold = True
    for news in stopcorona_headers_buisness[:10]:
        doc.add_paragraph(news, style='List Bullet')

    p = doc.add_paragraph()
    runner = p.add_run('Яндекс')
    runner.bold = True
    for news in yandex_annotation[:10]:
        doc.add_paragraph(news, style='List Bullet')

In [22]:
import docx
from docx.shared import RGBColor
doc = docx.Document()

paragraph_news(doc)
doc.save('новости.docx')


# Результат

In [23]:
from google.colab import files


In [24]:
files.download('новости.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>